In [50]:
import pandas as pd
import nltk
import gensim
from gensim import downloader
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from scipy.spatial import distance
import re
import numpy as np

glove_vectors = gensim.downloader.load('word2vec-google-news-300')
stop_words = set(stopwords.words('english'))

In [105]:
def Compute_ISBN_to_Title(df_merged):
    ISBN_to_Title = {}
    for row in df_merged.index:
        title = df_merged["Book-Title"][row]
        isbn = df_merged["ISBN"][row]
        ISBN_to_Title[isbn] = title
    return ISBN_to_Title

def merge_book_and_summary(df_summary, df_books):
    df_merged = pd.merge(df_summary, df_books, how='inner', on =['Book-Title'])
    df_merged = df_merged.drop(['Code1','Code2','Code3','Publisher', 'Year-Of-Publication_x', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'Book-Author_y', 'Year-Of-Publication_y'], axis=1)
    df_merged = df_merged[df_merged['Book-Title'] != 'Deathstalker Rebellion']
    return df_merged


df_books = pd.read_csv("Books.csv")
df_summary = pd.read_csv('booksummaries/booksummaries.txt', sep = '\t')
df_summary.columns = ['Code1', 'Code2', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Code3', 'Book-Summary']
ISBN_to_Title = Compute_ISBN_to_Title(df_merged)

/var/folders/v2/rwg0t7796jsbrq_0j88y5mk40000gn/T/ipykernel_25695/1247681004.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv("Books.csv")


In [106]:
def compute_Book2Vec(df_merged):
    title_dict = {}
    summary_dict = {}

    for row in df_merged.index:
        title = df_merged['Book-Title'][row]
        summary = df_merged['Book-Summary'][row]
        summary_nonum = re.sub(r'\d+', '', summary) #remove number
        tokenizer = RegexpTokenizer(r'\w+') #remove punctuation
        summary_tokens = tokenizer.tokenize(summary_nonum) 
        filtered_summary_tokens = [w for w in summary_tokens if not w.lower() in stop_words] #remove stop words

        summary_vectors = np.zeros(300)
    
        n = 0
        for summary_token in filtered_summary_tokens:
            try:
                summary_token_vec = glove_vectors[summary_token]
            except:
                summary_token_vec = np.zeros(300)
            summary_vectors += summary_token_vec
            n += 1
        summary_dict[title] = summary_vectors / n
    
    return summary_dict

df_merged = merge_book_and_summary(df_summary, df_books)
summary_dict = compute_Book2Vec(df_merged)

In [88]:
def compute_books_distance(summary_dict):
    books_count = len(summary_dict)
    books_distance = np.zeros(shape=(books_count, books_count))
    #print(books_count)
    summary_list = list(summary_dict)

    i = 0
    for key1 in summary_dict:
        j = 0
        summary_vec1 = summary_dict[key1]
        for key2 in summary_dict:
            summary_vec2 = summary_dict[key2]
            dst = distance.euclidean(summary_vec1, summary_vec2)
            if dst == 0: #same book:
                dst = 100000
            books_distance[i][j] = dst
            j+=1
        if i % 500 == 0:
            print(i)
    
        i+=1
    
    return books_distance

books_distance = compute_books_distance(summary_dict)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


In [104]:
def get_mostSimilar_title(title, summary_dict, books_distance):
    summary_list = list(summary_dict)
    input_distance = books_distance[summary_list.index(title)]
    closest_book_idx = np.argmin(input_distance)
    return summary_list[closest_book_idx]

def get_mostSimilar_isbn(isbn, summary_dict, books_distance, ISBN_to_Title):
    title = ISBN_to_Title[isbn]
    return get_mostSimilar_title(title, summary_dict, books_distance)

get_mostSimilar_isbn("345316413X", summary_dict, books_distance, ISBN_to_Title)

'Little Boy Blue'